In [2]:
import pandas as pd


In [3]:
PS_clean=pd.read_csv('googleplaystore(cleaned).csv',delimiter=';')

In [ ]:
PS_clean.head()

# Melihat Jumlah data terduplikat #

In [4]:
# Menginisialisasi daftar untuk menyimpan nama aplikasi duplikat dan unik
duplicate_apps = []
unique_apps = []

# Loop melalui nilai di kolom 'App' di dataframe
for name in PS_clean['App']:
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
    
# Menampilkan jumlah aplikasi duplikat dan beberapa contoh aplikasi duplikat
print('Jumlah data yang terduplikat:', len(duplicate_apps))
print('\n')
print('Aplikasi yang terduplikat:', duplicate_apps)

Jumlah data yang terduplikat: 1125


Aplikasi yang terduplikat: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software', 'MailChimp - Email, Marketing Automation', 'Crew - Free Messaging and Scheduling', 'Asana: organize team projects', 'Google Analytics', 'AdWords Express', 'Accounting App - Zoho Books', 'Invoice & Time Tracking - Zoho', 'join.me - Simple Meetings', 'Invoice 2go — Professional Invoices and Estimates', 'SignEasy | Sign and Fill PDF and other Documents', 'Quick PDF Scanner + OCR FREE', 'Genius Scan - PDF Scanner', 'Tiny Scanner - PDF Scanner App', 'Fast Scanner : Free PDF Scan', 'Mobile Doc Scanner (MDScan) Lite', 'Docs To Go™ Free Office Suite', 'OfficeSuite : Free Office + PDF Editor', 'Slack', 'QuickBoo

# Menampilkan nama dan jumlah aplikasi yang tidak terduplikat

In [5]:
# Membuat dictionary untuk menyimpan jumlah review maksimal per aplikasi
reviews_max = {}

# Loop melalui setiap baris di dataframe
for index, row in PS_clean.iterrows():
    name = row['App']
    n_reviews = float(row['Reviews'])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

# Menampilkan hasil dictionary
print('Expected Length:', len(reviews_max))
print('\n')
print('Contoh data non-duplikat:', list(reviews_max.items())[:15])

Expected Length: 7593


Contoh data non-duplikat: [('Coloring book moana', 974.0), ('U Launcher Lite – FREE Live Cool Themes, Hide Apps', 87510.0), ('Sketch - Draw & Paint', 215644.0), ('Pixel Draw - Number Art Coloring Book', 967.0), ('Paper flowers instructions', 167.0), ('Smoke Effect Photo Maker - Smoke Editor', 178.0), ('Infinite Painter', 36815.0), ('Garden Coloring Book', 13791.0), ('Kids Paint Free - Drawing Fun', 121.0), ('Text on Photo - Fonteee', 13880.0), ('Name Art Photo Editor - Focus n Filters', 8788.0), ('Tattoo Name On My Photo Editor', 44829.0), ('Mandala Coloring Book', 4326.0), ('3D Color Pixel by Number - Sandbox Art Coloring', 1518.0), ('Learn To Draw Kawaii Characters', 55.0)]


# Menyaring data yang terduplikasi sebelumnya berdasarkan kolom review #

In [6]:
# Membuat list untuk menyimpan baris yang tidak terduplikat
PS_clean_unique = []

# Loop melalui setiap baris di dataframe untuk menyaring data duplikat
for index, row in PS_clean.iterrows():
    name = row['App']
    n_reviews = float(row['Reviews'])
    
    # Jika jumlah review sesuai dengan yang ada di reviews_max, tambahkan ke list
    if reviews_max[name] == n_reviews:
        PS_clean_unique.append(row)

# Mengubah list menjadi dataframe
PS_clean_unique_df = pd.DataFrame(PS_clean_unique)

# Menampilkan hasil
print('Jumlah aplikasi unik:', PS_clean_unique_df['App'].nunique())
print('\nContoh data tanpa duplikat:')
print(PS_clean_unique_df.head())



Jumlah aplikasi unik: 7593

Contoh data tanpa duplikat:
                                                 App        Category  Rating  \
1  U Launcher Lite – FREE Live Cool Themes, Hide ...  ART_AND_DESIGN     4.7   
2                              Sketch - Draw & Paint  ART_AND_DESIGN     4.5   
3              Pixel Draw - Number Art Coloring Book  ART_AND_DESIGN     4.3   
4                         Paper flowers instructions  ART_AND_DESIGN     4.4   
5            Smoke Effect Photo Maker - Smoke Editor  ART_AND_DESIGN     3.8   

   Reviews                   Genres  
1    87510             Art & Design  
2   215644             Art & Design  
3      967  Art & Design;Creativity  
4      167             Art & Design  
5      178             Art & Design  


In [9]:
# Menghapus duplikat berdasarkan kolom 'App' dan menyimpan hanya baris pertama (yaitu jumlah ulasan tertinggi)
PS_clean_unique_df = PS_clean_unique_df.drop_duplicates(subset='App', keep='first')

In [11]:
PS_clean_unique_df[PS_clean_unique_df['App'] == 'Google My Business']

,App,Category,Rating,Reviews,Genres
182,Google My Business,BUSINESS,4.4,70991,Business


# Mengecek dan menghapus data yang mengandung non-Ascii>3

In [13]:
def is_english(string):
    non_ascii = 0  # Menginisialisasi penghitung karakter non-ASCII
    
    for character in string:  # Mengiterasi setiap karakter bentuk string
        if ord(character) > 127:  # Memeriksa apakah karakter adalah non-ASCII
            non_ascii += 1  # Jika ya, tambahkan ke penghitung
    
    if non_ascii > 3:  # Jika lebih dari 3 karakter adalah non-ASCII
        return False  # Menganggap string tersebut bukan bahasa Inggris
    else:
        return True  # Jika 3 atau lebih sedikit karakter adalah non-ASCII, menganggap string tersebut bahasa Inggris

print(is_english('Docs To Go™ Free Office Suite'))  
print(is_english('Instachat 😜')) 

True
True


In [14]:
# List untuk menyimpan aplikasi yang namanya dalam bahasa Inggris
android_english = []

# Loop melalui setiap baris di dataframe untuk memeriksa apakah nama aplikasi dalam bahasa Inggris
for index, row in PS_clean_unique_df.iterrows():
    name = row['App']
    if is_english(name):
        android_english.append(row)

# Mengubah list menjadi dataframe
android_english_df = pd.DataFrame(android_english)

In [15]:
print('Jumlah aplikasi dalam bahasa Inggris:', android_english_df.shape[0])
print('\nContoh data aplikasi dalam bahasa Inggris:')
print(android_english_df.head())

Jumlah aplikasi dalam bahasa Inggris: 7565

Contoh data aplikasi dalam bahasa Inggris:
                                                 App        Category  Rating  \
1  U Launcher Lite – FREE Live Cool Themes, Hide ...  ART_AND_DESIGN     4.7   
2                              Sketch - Draw & Paint  ART_AND_DESIGN     4.5   
3              Pixel Draw - Number Art Coloring Book  ART_AND_DESIGN     4.3   
4                         Paper flowers instructions  ART_AND_DESIGN     4.4   
5            Smoke Effect Photo Maker - Smoke Editor  ART_AND_DESIGN     3.8   

   Reviews                   Genres  
1    87510             Art & Design  
2   215644             Art & Design  
3      967  Art & Design;Creativity  
4      167             Art & Design  
5      178             Art & Design  


In [16]:
#Mengecek salah satu data yang terduplikasi sebelumnya
android_english_df[android_english_df['App'] == 'Instagram']


,App,Category,Rating,Reviews,Genres
2325,Instagram,SOCIAL,4.5,66577446,Social


In [17]:
len(android_english_df)

7565

# Memastikan sudah tidak ada terduplikasi###

In [18]:
# Menginisialisasi daftar untuk menyimpan nama aplikasi duplikat dan unik
duplicate_apps2 = []
unique_apps2 = []

# Loop melalui nilai di kolom 'App' di dataframe
for name in android_english_df['App']:
    if name in unique_apps2:
        duplicate_apps2.append(name)
    else:
        unique_apps.append(name)
    
# Menampilkan jumlah aplikasi duplikat dan beberapa contoh aplikasi duplikat
print('Number of duplicate apps:', len(duplicate_apps2))
print('\n')
print('Examples of duplicate apps:', duplicate_apps2[:15])

Number of duplicate apps: 0


Examples of duplicate apps: []


# Menyimpan dalam format xlsx dan csv


In [19]:
android_english_df.to_excel('android_english_apps.xlsx', index=False, engine='openpyxl')


In [20]:
android_english_df.to_csv('android_english_apps.csv',index=False,sep=',')